In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from datetime import date 
import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
import itertools
import math

from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
data =  pd.read_csv("demo_try.csv", parse_dates=True)
data.head()

In [ ]:
data_new = data.loc[:,['Name','Date','Quantity','Item : Internal ID']]
data_new.rename(columns={'Item : Internal ID': 'Product_Id', 'Name':'Customer_Name'}, inplace=True)

In [ ]:
result = data_new.groupby(['Customer_Name', 'Product_Id', 'Date', 'Quantity']).size().reset_index()
# result = result.count() #result['Customer_Name'].value_counts()
result.head(6)

In [ ]:
all_customer = set(data_new['Customer_Name'])

In [ ]:
def lock_customer_details(customer_name, result):
    data_final = result[result['Customer_Name'] == customer_name]
    return data_final

def prod_wise_customer(prod_wise_cus, gr_wise_pr):
    final_ = prod_wise_cus.get_group(gr_wise_pr)
    final_ = final_.loc[:,['Date','Quantity']]
    final_["Date"]= pd.to_datetime(final_["Date"], dayfirst=True)
    final_.set_index('Date', inplace=True)
    final_.fillna(0,inplace=True)
    final_ =final_['Quantity'].resample('D').sum()
    print(final_)

In [ ]:
for customer in all_customer:
    customer_info = lock_customer_details(customer, result)
    prod_wise_cus = customer_info.groupby(['Product_Id'])
    for gr_wise_pr in prod_wise_cus.groups.keys():
        final_df = prod_wise_customer(prod_wise_cus, gr_wise_pr)
        print(final_df)
        break
    # print(customer_info)
    break

In [ ]:
def Simple_Exponential_Smooting(train_data, n_period):
    model_simple_es = SimpleExpSmoothing(train_data).fit()
    predictions_simple_es = model_simple_es.forecast(n_period)
    test_pred_ses = predictions_simple_es[-n_period:]
    predictions_ses_final = test_pred_ses.mask(test_pred_ses<0,0).apply(np.ceil)
    return predictions_ses_final

In [ ]:
print(Simple_Exponential_Smooting(final_df, 1))

In [ ]:
import numpy as np
from datetime import datetime, timedelta
from hmmlearn import hmm

In [ ]:
# Hidden Markov Model initialization
model = hmm.MultinomialHMM(n_components=10)

In [ ]:
X = [[5, 9, 14, 55], [66, 9, 15, 5]]

In [ ]:
# Fit the model to the data
model.fit(X)

In [ ]:
next_purchase = model.predict([[66, 9, 15, 5]])  # Use February data as input
print(f"Predicted next purchase: Product {next_purchase[0]}")

### Code Updated

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from datetime import date 
import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
import itertools
import math

from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
data =  pd.read_csv("demo_try.csv", parse_dates=True)
data.head()

In [ ]:
data_new = data.loc[:,['Customer Name','Date','Quantity','Item : Internal ID']]
data_new.rename(columns={'Item : Internal ID': 'Product_Id', 'Customer Name':'Customer_Name'}, inplace=True)

In [ ]:
result = data_new.groupby(['Customer_Name', 'Product_Id', 'Date', 'Quantity']).size().reset_index()
result

In [ ]:
all_customer = set(data_new['Customer_Name'])
all_product = set(result['Product_Id'])

In [ ]:
def set_all_attribute(resample_type):
    if resample_type == "M":
        thresold_nonzero = 15
        test_period = 8
    elif resample_type == "Q":
        thresold_nonzero = 20
        test_period = 3
    elif resample_type == "Y":
        thresold_nonzero = 40
        test_period = 1
    return thresold_nonzero, test_period

In [ ]:
def Double_Exponential_Smoothing(train_data, test_data, n_period, flag, resample_type):
    double_model_es = ExponentialSmoothing(train_data, trend='add').fit()
    predictions_double_es = double_model_es.forecast(n_period)
    test_pred_des = predictions_double_es[-n_period:]
    predictions_des_final = test_pred_des.mask(test_pred_des<0,0).apply(np.ceil)
    predictions_des_final_raw = test_pred_des.mask(test_pred_des<0,0)
    if flag == 1:
        mae_des =np.round(np.mean(np.abs(test_data - predictions_des_final)),2)
        return "Double Exponential Smooting", mae_des
    else:
        if resample_type == "M":
            std_des = np.std(train_data[-24:])    #predictions_des_final) -24
            if std_des > 0:
                ci_des = 1.28 * std_des           ### 80% Confidence Level
            else:
                ci_des = (predictions_des_final * .2)
            weight_list = [4,4,4,5,5,8,8,10,10,13,14,15] #= 100
            # weight_list = [12,18,20,25,25]
            # weight_list = [10,20,35,35]
            historic_value = list(train_data[-12:])
            weighted_avg = np.round(np.average( historic_value, weights = weight_list), 2)
            weighted_avg = np.maximum(np.ceil(weighted_avg), 0) #np.ceil(weighted_avg)
            dict_sheet = {}
            conf_int_des=pd.DataFrame({
                                       'lower': np.floor(predictions_des_final - ci_des) ,
                                       'upper':np.ceil(predictions_des_final + ci_des)}, 
                                        index=predictions_des_final.index)
            
            if (int(predictions_des_final[0]) > int(weighted_avg * .7)) and (int(predictions_des_final[0]) <  int(weighted_avg * 1.4)):    
                dict_sheet['Forecasted_value'] = predictions_des_final
                dict_sheet['Lower'] = conf_int_des.lower.mask(conf_int_des.lower<0,0)
                dict_sheet['Upper'] = conf_int_des.upper.mask(conf_int_des.upper<0,0)
                return dict_sheet
            else:
                forecasted_avg = np.ceil(weighted_avg)
                forecasted_min = np.floor(forecasted_avg - (forecasted_avg * .2))
                forecasted_max = np.ceil(forecasted_avg + (forecasted_avg * .25))

                predictions_des_final[predictions_des_final.index[0]] = forecasted_avg
                conf_int_des.lower[predictions_des_final.index[0]] = forecasted_min
                conf_int_des.upper[predictions_des_final.index[0]] = forecasted_max

                dict_sheet['Forecasted_value'] = predictions_des_final.mask(predictions_des_final<0,0)
                dict_sheet['Lower'] = conf_int_des.lower.mask(conf_int_des.lower<0,0)
                dict_sheet['Upper'] = conf_int_des.upper.mask(conf_int_des.upper<0,0)
                return dict_sheet
        else:
            std_des_raw = np.std(train_data[-24:])   #predictions_des_final_raw)
            if std_des_raw > 0:
                ci_des_raw = 1.28 * std_des_raw      ### 80% Confidence Level
            else:
                ci_des_raw = (predictions_des_final_raw * .2)
            conf_int_des=pd.DataFrame({
                                      'lower': np.floor(predictions_des_final_raw - ci_des_raw),
                                      'upper':np.ceil(predictions_des_final_raw + ci_des_raw)},
                                       index=predictions_des_final_raw.index)
            dict_sheet = {}
            dict_sheet['Forecasted_value'] = np.round(predictions_des_final_raw)
            dict_sheet['Lower'] = conf_int_des.lower.mask(conf_int_des.lower<0,0)
            dict_sheet['Upper'] = conf_int_des.upper.mask(conf_int_des.upper<0,0)
            return dict_sheet

In [ ]:
def ARIMA(train_data, test_data, n_period, flag, resample_type, check_period):
  if resample_type == "Q": 
        simple_diff = True
        check_period = 3
  else: 
        simple_diff = False
        # check_period = int(np.round(len(train_data)*.1))
        check_period = check_period
  p = d = q = range(1, 3)
  pdq_a = list(itertools.product(p, d, q))
  minimum_mae = math.inf
  for param in pdq_a:
        modd = sm.tsa.statespace.SARIMAX(train_data, order = param, simple_differencing=simple_diff)                                  
        try:
            resultsss = modd.fit(disp=False)
            predict_first = resultsss.get_prediction(end = modd.nobs-1)
            pred_arima_mean = np.round(predict_first.predicted_mean)
            pred_arima_masked = pred_arima_mean.mask(pred_arima_mean<=0,0)
            # print("pred_arima_mean", param, pred_arima_masked[-12:], train_data[-12:])
            mae_arima_first =np.round(np.mean(np.abs(train_data[-check_period:] - pred_arima_masked[-check_period:])),2)
            if mae_arima_first < minimum_mae:
                    minimum_mae = mae_arima_first
                    minimum_param_arima = param
        except:
            continue
  moddel_arima = sm.tsa.statespace.SARIMAX(train_data, order = minimum_param_arima, simple_differencing = simple_diff)
  result_arima = moddel_arima.fit(disp = False)
  predict = result_arima.get_forecast(steps = n_period)
  pred_arima = predict.predicted_mean[-n_period:]
  predictions_arima_final = pred_arima.mask(pred_arima<0,0).apply(np.ceil)
  predictions_arima_final_raw = pred_arima.mask(pred_arima<0,0)
  max_inside_history = max(train_data)
  if flag == 1:
        mae_arima =np.round(np.mean(np.abs(test_data - predictions_arima_final)),2)
        return "ARIMA", mae_arima
  else:     
        if resample_type == "M":
            std_arima = np.std(train_data[-24:])     # predictions_arima_final
            if std_arima > 0:
                ci_arima = 1.28 * std_arima          ### 80% Confidence Level
            else:
                ci_arima = (predictions_arima_final * .15)
            dict_sheet = {}
            conf_int_arima=pd.DataFrame({
                                        'lower': np.floor(predictions_arima_final - ci_arima),
                                        'upper': np.ceil(predictions_arima_final + ci_arima)}, 
                                         index=predictions_arima_final.index)      
            minimum_forecasted_val = conf_int_arima.lower.mask(conf_int_arima.lower<0,0)
            maximum_forecasted_val = conf_int_arima.upper.mask(conf_int_arima.upper<0,0)
            weight_list = [3,3,4,5,5,7,8,10,10,15,15,15] #= 100
            # weight_list = [12,18,20,25,25]
            # weight_list = [10,20,35,35]
            historic_value = list(train_data[-12:])
            weighted_avg = np.round(np.average( historic_value, weights = weight_list), 2)
            weighted_avg = np.maximum(np.ceil(weighted_avg), 0)     #np.ceil(weighted_avg) 
            if minimum_forecasted_val[0] > max_inside_history or np.ceil((.7 * weighted_avg)) > maximum_forecasted_val[0]:
                forecasted_avg = np.ceil(weighted_avg)
                forecasted_min = np.floor(forecasted_avg - (forecasted_avg * .2))
                forecasted_max = np.ceil(forecasted_avg + (forecasted_avg * .2))

                predictions_arima_final[predictions_arima_final.index[0]] = forecasted_avg
                conf_int_arima.lower[predictions_arima_final.index[0]] = forecasted_min
                conf_int_arima.upper[predictions_arima_final.index[0]] = forecasted_max

                dict_sheet['Forecasted_value'] = predictions_arima_final.mask(predictions_arima_final<0,0)
                dict_sheet['Lower'] = conf_int_arima.lower.mask(conf_int_arima.lower<0,0)
                dict_sheet['Upper'] = conf_int_arima.upper.mask(conf_int_arima.upper<0,0)
            else:
                dict_sheet['Forecasted_value'] = predictions_arima_final.mask(predictions_arima_final<0,0)
                dict_sheet['Lower'] = conf_int_arima.lower.mask(conf_int_arima.lower<0,0)
                dict_sheet['Upper'] = conf_int_arima.upper.mask(conf_int_arima.upper<0,0)
            return dict_sheet
        else:
            std_arima_raw = np.std(train_data[-24:])    #predictions_arima_final_raw)
            if std_arima_raw > 0:
                ci_arima_raw = 1.28 * std_arima_raw     ### 80% Confidence Level
            else:
                ci_arima_raw = (predictions_arima_final_raw * .2)
            conf_int_arima=pd.DataFrame({
                                      'lower': np.floor(predictions_arima_final_raw - ci_arima_raw),
                                      'upper':np.ceil(predictions_arima_final_raw + ci_arima_raw)},
                                       index=predictions_arima_final_raw.index)
            dict_sheet = {}
            dict_sheet['Forecasted_value'] = np.round(predictions_arima_final_raw)
            dict_sheet['Lower'] = conf_int_arima.lower.mask(conf_int_arima.lower<0,0)
            dict_sheet['Upper'] = conf_int_arima.upper.mask(conf_int_arima.upper<0,0)
            return dict_sheet

In [ ]:
def check_method_first_phase(percentage_of_nonzero, product_sales_data):
    resample_type = "M" #"M"
    product_len = len(product_sales_data)
    thresold_nonzero, test_period = set_all_attribute(resample_type)
    if percentage_of_nonzero >= thresold_nonzero:
        train_data = product_sales_data[:-test_period]
        test_data = product_sales_data[-test_period:]
    else:
        product_len = train_data = test_data = None
    return product_len, train_data, test_data, thresold_nonzero

In [ ]:
def lock_product_details(product, result):
    data_product = result[result['Product_Id'] == product]
    final_ = data_product.loc[:,['Date','Quantity']]
    final_["Date"]= pd.to_datetime(final_["Date"], dayfirst=True)
    final_.set_index('Date', inplace=True)
    final_.fillna(0,inplace=True)
    final_ =final_['Quantity'].resample('M').sum()

    data = [{'Quantity': 0}]
    today = datetime.date.today()
    end_date = today.replace(day=1) - datetime.timedelta(days=1)
    start_date = end_date + relativedelta( years=-5 )
    df = pd.DataFrame( data, index=pd.date_range( start=start_date, end=end_date, freq="M" ) )
    df.index.rename( 'Date', inplace=True )
    ddf = df.squeeze()
    final__ = ddf.add( final_, fill_value = 0 )
    return final__

In [ ]:
def method_check_final(model_1, mae_1, model_2, mae_2):
    if mae_1 < mae_2:
        return model_1
    else:
        return model_2

In [ ]:
def check_method_based_length( train_data, test_data, baseline_value):
    flag = 1
    test_period = len(test_data)
    resample_type = "M"
    model_1, mae_1 = ARIMA(train_data, test_data, test_period, flag, resample_type, baseline_value)
    model_2, mae_2 = Double_Exponential_Smoothing(train_data, test_data, test_period, flag, resample_type)
    final_method = method_check_final(model_1, mae_1, model_2, mae_2)
    return final_method

In [ ]:
def all_function_forecast(train_data, method, check_period):
    test_data = 0
    flag = 2
    if method == "ARIMA": 
        forecast_result = ARIMA(train_data, test_data, 8, flag, "M", check_period)
    elif method == "Double Exponential Smooting":
        forecast_result = Double_Exponential_Smoothing(train_data, test_data, 8, flag, "M")
    return forecast_result, method

In [ ]:
dict_sheet_2 = {}
for ind, product in enumerate(all_product):
    try:
        prod_sales_data = lock_product_details(product, result)
        non_zero_length = np.count_nonzero(prod_sales_data)
        percentage_of_nonzero = (non_zero_length / len(prod_sales_data)) * 100
        product_len, train_data, test_data, thresold_nonzero = check_method_first_phase(percentage_of_nonzero, prod_sales_data)
        dict_sheet_2[product] = {}
        if product_len != None:
            # method = check_method_based_length( train_data, test_data, int(len(test_data)) )
            # method = method.strip()
            method = "ARIMA"
            forecast_result, method_r = all_function_forecast(prod_sales_data, method, int(len(test_data)))
            dict_sheet_2[product]['Historic Data'] = prod_sales_data[-24:].to_string() #-25  [-9:]
            dict_sheet_2[product]['Forecasted_Value'] = forecast_result["Forecasted_value"].to_string()
            dict_sheet_2[product]['Min Qty'] = forecast_result["Lower"].to_string()
            dict_sheet_2[product]['Max Qty'] = forecast_result["Upper"].to_string()
            # dict_sheet_2[product]['Method'] = method
        # if ind==100:
            # print(percentage_of_nonzero, non_zero_length, method)
            # break
    except:
        continue
pd.DataFrame.from_dict(dict_sheet_2, orient='index').to_csv('Product_wise_forecast_monthly.csv', index_label="Product_ID")